# Обучение моделей на SQuAD с смержиными текстами

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Доступные датасеты:

In [2]:
dataset_names = ['initial', 'same topic', 'different topics']
dataset_paths =['/content/drive/MyDrive/QA_training/merged datasets/squad/initial_train.csv',
                '/content/drive/MyDrive/QA_training/merged datasets/squad/merged_by_2_train.csv',
                '/content/drive/MyDrive/QA_training/merged datasets/squad/random_merged_by_2_train.csv']

Модели:

In [4]:
model_paths = ['/content/drive/MyDrive/QA_training/models/squad_models/merged_by_1_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/merged_by_2_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/randon_merged_by_2_bert']

new_model_names = ['merg_by_1_5_epochs', 'merg_by_2_5_epochs', 'random_merg_by_2_5_epochs']

Обучение

In [6]:
!pip install import-ipynb
%cd '/content/drive/My Drive/QA_training/notebooks'
import import_ipynb
from my_functions import *

/content/drive/My Drive/QA_training/notebooks
importing Jupyter notebook from my_functions.ipynb
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gx0fe4fa
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-gx0fe4fa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 9.9MB/s 
     |████████████████████████████████| 890kB 58.4MB/s 
  Created wheel for transformers: filename=transformers-4.2.0.dev0-cp36-none-any.whl size=1680279 sha256=10542b9d926b0fff23b40390b33cf08b2c6de53a0304ea2ce1ec83bfad9cbb0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-6j0qr_po/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=01f30b29c429477fa00b38c5754f5452010791637d1f53137cb9b9

In [7]:
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [8]:
number = 0

In [9]:
train_csv_path = dataset_paths[number]
print('Dataset path:', train_csv_path)

train_dataset = prepare_and_merge_data(train_csv_path, tokenizer, 'squad', text_num=number+1)


Dataset path: /content/drive/MyDrive/QA_training/merged datasets/squad/initial_train.csv


In [10]:
dir = "/content/drive/MyDrive/QA_training/models/squad_models/"+ new_model_names[number]

print('Importing model path:', model_paths[number])
print('New model path:', dir)
model = BertForQuestionAnswering.from_pretrained(model_paths[number])
#model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")
#model =  BertForQuestionAnswering.from_pretrained(dir+'/checkpoint-26000')

Importing model path: /content/drive/MyDrive/QA_training/models/squad_models/merged_by_1_bert
New model path: /content/drive/MyDrive/QA_training/models/squad_models/merg_by_1_5_epochs


In [11]:
training_args = TrainingArguments(
    output_dir=dir,          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir=dir+'/logs',            # directory for storing logs
    logging_steps=250,
    prediction_loss_only=False,
    label_names = ["start_positions", "end_positions"],
    overwrite_output_dir = True,
    learning_rate = 2e-5
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    #eval_dataset=val_dataset,            # evaluation dataset

)

In [12]:
trainer.train()
#trainer.train(dir+'/checkpoint-13000')
trainer.save_model(dir)

Step,Training Loss
250,0.636268
500,0.536148
750,0.539814
1000,0.490223
1250,0.485752
1500,0.446781
1750,0.441508
2000,0.439842
2250,0.395405
2500,0.406255
